# Nomes

* namespaces e escopos
    * exec e eval?
* naming and binding
    * contador de referencias e o comando del
    * modificações ao deletar (ou a eficiência do comando del)
    * singletons implícitos (chars e ints pequenos)
    * garbage collection
* Bonus: if name == main

## Nomes e vínculos

### Mecanismo de nomeação em Python

Quando atribuímos algum objeto a um **nome** em Python, usando o operador de atribuição ```=```, não estamos propriamente pegando o conteúdo atribuído e copiando para um local da memória reservado àquele nome. Ocorre o inverso: o Python faz com que aquele nome aponte para o local da memória onde está o conteúdo atribuído. Em outras palavras, a atribuição por si só não copia dados. Outro termo usado na documentação oficial para designar o nome é **identificador**.

Já o ponteiro que aponta para o objeto, ou seja, o valor que está  estocado na memória por trás deste nome, é chamado de **referência**.

O ato de estabelecer essa conexão entre nomes e referências a objetos no contexto de um namespace é chamado de **vínculo** ou vinculação.

(Obs.: em alguns textos pode-se encontrar uma diferença sutil entre identificador e nome, sendo o primeiro mais usado para se referir ao tipo de token na análise léxica, e o segundo empregado para o uso efetivo do identificador na memória, em tempo de execução.)

Esse sistema de referência sem cópia é um elemento importante para dar maior eficiência da linguagem Python. Nessa linha, algo parecido ocorre quando passamos um nome no argumento da função: não é o objeto em si que é copiado e passado, mas apenas uma referência àquele objeto.

### Criação e destruição de vínculos

Há vários constructos da linguagem que constróem vínculos, ora de maneira mais maneira explícita pelo usuário (epressões de atribuição, definições de funções e classes), ora de maneira implícita pelo próprio Python. Apresentamos abaixo uma lista dos comandos que criam vínculos. (No presente material, pretendemos abordar todos esses itens, exceto o loop ```for```, que é amplamente coberto em materiais introdutórios.)
* expressões de atribuição tais como ```x = 1```, que vimos acima;
* quando funções são chamadas, os valores passados a cada argumento são vinculados ao nome do argumento, como se fosse uma atribuição (*pass-by-assignment*);
* a definição de uma classe vincula seu nome a uma instância do tipo ```type``` (que é única);
* a definição de uma função vincula seu nome a uma instância do tipo ```function```;
* no comando ```import```, que vincula os nomes do módulo importado, ou ```from [modulo] import```, que vincula todos os nomes exceto aqueles começando em ```_```;
* instruções de tipagem e listas de parâmetros de tipo
* identificadores usados em:
  - um loop ```for```;
  - depois de ```as``` com as instruções ```with```, ```except```, ```except*```, e em um ```case``` de ```match```; e
  - numa captura em um ```case``` de ```match```.

Para desfazer o vínculo entre um objeto e um nome no escopo local, usa-se a instrução ```del [nome]``` (Notar: para fazer isso, o comando ```del``` também cria temporariamente um vínculo). O fechamento do escopo local onde o nome foi vinculado também destrói o vínculo. Veremos o que isso significa mais adiante neste tópico. Quando usada com a instrução ```global```, ```del``` em vez disso remove o vínculo feito ao mesmo nome no escopo global. Por fim, a interrupção do funcionamento do interpretador de Python destrói todos os vínculos remanescentes, encerrando a execução do Python.

Tentar usar um nome que não está vinculado a nenhum objeto levanta o erro ```NameError```:

In [21]:
x = 1
del x
print(x)

NameError: name 'x' is not defined

Ao atuar primariamente na remoção do vínculo, o comando ```del``` pode herdar as eficiências da forma como Python lida com a nomeação, pois não é necessário limpar ou sobrescrever a representação binária do objeto na memória, apenas esquecer seu endereço - ou seja, tornar impossível ao interpretador do Python encontrar o objeto. Uma vez liberado, aquele trecho de memória pode ser livremente usado para outra tarefa.

(Obs.: implementações específicas do Python podem sobrescrever parte do conteúdo do objeto por motivos de segurança, contagem de referências, flags do coletor de lixo, reaproveitamento para o alocador de memória, entre outras - ver seção sobre CPython adiante para algumas dessas especificidades.)

### Sinônimos

Ao atribuirmos um nome a outro nome já existente, como no código abaixo, o Python vincula o novo nome à mesma referência por trás do nome original. Dois nomes vinculados a um exato mesmo objeto na memória são por vezes chamados de **sinônimos**.

O operador ```is``` verifica se dois nomes são sinônimos. Isto é, verifica se o endereço na memória (objeto) para o qual dois nomes apontam é o mesmo. ```x is y``` é equivalente a ```id(x) == id(y)```. Isso é diferente do operador ```==```, que compara se o valor de dois objetos é igual. Vejamos um exemplo que ilustra essa diferença:

In [10]:
# exemplo 1: objetos sinônimos
x = list('zzz') # criamos uma instância do tipo list
y = x # criamos um sinônimo para x: y
print(f'x e y têm mesmo valor? {x == y}')
print(f'x e y são sinônimos? {x is y}')

# exemplo 2: objetos de mesmo valor
w = list('nno') # criamos outra instância do tipo list
z = list('nno') # uma terceira instância (outro objeto), de mesmo valor
print(f'w e z têm mesmo valor? {w == z}')
print(f'w e z são sinônimos? {w is z}')

x e y têm mesmo valor? True
x e y são sinônimos? True
w e z têm mesmo valor? True
w e z são sinônimos? False



O operador ```is``` pode permitir grandes ganhos de eficiência, mas é necessário atentar para o fato de que ele pode apresentar uma série de nuances quanto a algumas categorias de objetos (singletons, comparações de tipo) e comportamentos que dependem da implementação (internamentos, imortais).

Além de meros nomes, é também possível que haja expressões de cada lado de ```is```, que continua funcionando da mesma forma (i.e., compara se as duas expressões resultam exatamente no mesmo objeto). No entanto, isso pode levar a comportamentos inesperados, pelos mesmos motivos, de modo que é recomendável comparar apenas nomes até que o usuário adquira mais conhecimento deste operador e das possíveis implicações específicas à implementação.

Uma exceção a este conselho que é comumemente introduzida em nível iniciante é a comparação de tipo. Trata-se de expressões da seguinte forma:

In [20]:
t = type(1)
print(t is int)

True


Bem, por que isso acontece? Sabemos que ```type(1)``` retorna ```int```, mas como eu sei que *este* ```int```, vinculado ao nome ```t```, é *o mesmo* ```int``` da definição da classe ```int```? Isso ocorre porque, em Python, todo objeto do tipo ```type``` só admite uma única instância. Ou seja, todo nome que apontar para um dado objeto do tipo ```type```, de um dado valor, sempre apontará para este mesmo objeto. Algo semelhante ocorre, por exemplo, com ```None```, embora este último seja tecnicamente um singleton:

In [ ]:
# duas instanciações: criam objetos diferentes
a = []
b = []
print(a is b)

# singleton: é sempre o mesmo objeto
c = None
d = None
print(c is d)

False
True


Outro jeito de definir sinônimos é a chamada atribuição encadeada (*chained assignment*), que é feita com a seguinte sintaxe: ```nome1 = nome 2 [= nome3...] = objeto```.

Se, depois de criarmos um sinônimo por meio de um segundo nome, atribuirmos um novo objeto ao nome original, estabelecemos para o nome original um novo vínculo, abandonando a primeira referência e substituindo por uma nova. No exemplo abaixo, criamos sinônimos por atribuição encadeada e depois vinculamos um dos nomes a outro objeto:

In [17]:
# antes
x = y = list('+-+')
print('Antes da segunda atribuição')
print(f'Valor apontado por y: {y}') 
print(f'Valor apontado por x: {x}')

# depois
x = list('-+-')
print('Depois da segunda atribuição')
print(f'Valor apontado por y: {y}') # y preserva a referência ao objeto original
print(f'Valor apontado por x: {x}') # x passa a apontar para este novo objeto

Antes da segunda atribuição
Valor apontado por y: ['+', '-', '+']
Valor apontado por x: ['+', '-', '+']
Depois da segunda atribuição
Valor apontado por y: ['+', '-', '+']
Valor apontado por x: ['-', '+', '-']


Notar que checagens semelhantes podem ser feitas usando os operadores ```is``` e ```==```, ou ainda a função ```id()```.

## Mutabilidade

### O conceito

**REDIGIR**

In [4]:
x = 1 # x aponta para um objeto int(1)
y = x # y aponta para o mesmo objeto int(1)
x += 1 # como int é imutável, é instanciado um novo objeto int(2)
print(f'Valor apontado por x: {x}') # x passa a apontar para este novo objeto int(2)
print(f'Valor apontado por y: {y}') # y preserva a referência ao objeto original int(1)

Valor apontado por x: 2
Valor apontado por y: 1


 Notar, ainda, que o código acima funciona dessa forma porque um número inteiro é um imutável, levando o Python a criar uma nova instância de objeto ao realizarmos a atribuição ```+=```. O mesmo pode não ocorrer com objetos mutáveis. Vejamos a diferença no caso do código abaixo. Como estamos modificando o valor do objeto já atribuído, nenhuma nova instância é criada, preservando em ambos os identificadores a referência original.

In [5]:
x = [1]
y = x
x += [1]
print(f'Valor apontado por x: {x}')
print(f'Valor apontado por y: {y}')

Valor apontado por x: [1, 1]
Valor apontado por y: [1, 1]


### Exceção à mutabilidade

tuplas com listas

### Erros frequentes na manipulação de objetos mutáveis

### Copiando mutáveis

## Alguns detalhes da implementação CPython

### Contador de referências

Todo valor em CPython, quando é criado, recebe uma sequência de bits logo em seu começo que trata única e exclusivamente de contar o número de vínculos que atualmente levam àquele objeto. Sempre que adicionamos uma referência ou um vínculo ao mesmo objeto, o contador é acrescido uma unidade. Sempre que executamos a instrução ```del [nome]```, o contador de referências é decrementado em uma unidade. Sempre que um escopo é fechado, as vinculações correspondentes àqueles nomes são subtraídas.

Quando o contador de referências de um objeto chega a ```0```, ele não é mais encontrável no programa, pois não há como saber onde ele se encontra na memória. Tal objeto estará então pronto para ser coletado pelo coletor de lixo automático do CPython (*garbage collector*), para fins de liberação de memória.

No código abaixo, usamos as bibliotecas ```sys``` e ```ctypes``` para obter acesso direto ao conteúdo do objeto ```obj1``` na memória. Não nos preocuparemos aqui com os detalhes dessas funcionalidades, que não são tema deste tópico. Tudo que precisamos saber é que este código imprime os bits do objeto. Notar que o primeiro byte do objeto estoca o número $1$ em binários (```00000001```), justamente porque existe um único vínculo a este objeto, envolvendo o nome ```obj1```.

In [ ]:
import ctypes
import sys

obj1 = 'a' * 500 # cria o objeto (uma string grande)
addr = id(obj1) # pega o endereço do objeto na memória
n_bytes = sys.getsizeof(obj1) # pega o número de bytes ocupado pelo objeto
ptr = ctypes.cast(addr, ctypes.POINTER(ctypes.c_uint8)) # cria um pointer de C para o endereço do objeto
lista_bits = ''.join(f'{ptr[i]:08b} ' for i in range(n_bytes))[:-1].split(' ') # lê cada byte do objeto e estoca em uma lista de strings

# loop para visualizar n bytes em uma linha
n = 8
visualizacao = ''
espaco = '\n '
for i, byte in enumerate(lista_bits):
    tipo_de_espaco = bool((i+1) % n)
    visualizacao += (byte + espaco[tipo_de_espaco])
print(visualizacao)

00000001 00000000 00000000 00000000 00000000 00000000 00000000 00000000
01110000 10011111 10010101 11000111 11111001 01111111 00000000 00000000
11110100 00000001 00000000 00000000 00000000 00000000 00000000 00000000
01100101 00001010 01011000 10101101 00010100 00100011 10011000 10100011
01100101 00000000 00000000 00000000 00000000 00000000 00000000 00000000
01100001 01100001 01100001 01100001 01100001 01100001 01100001 01100001
01100001 01100001 01100001 01100001 01100001 01100001 01100001 01100001
01100001 01100001 01100001 01100001 01100001 01100001 01100001 01100001
01100001 01100001 01100001 01100001 01100001 01100001 01100001 01100001
01100001 01100001 01100001 01100001 01100001 01100001 01100001 01100001
01100001 01100001 01100001 01100001 01100001 01100001 01100001 01100001
01100001 01100001 01100001 01100001 01100001 01100001 01100001 01100001
01100001 01100001 01100001 01100001 01100001 01100001 01100001 01100001
01100001 01100001 01100001 01100001 01100001 01100001 01100001 0

Agora vamos encapsular uma parte deste código na função ```bits()```, que retorna os bits do objeto. Criamos também uma função de comparação para imprimir verticalmente o conteúdo de dois objetos de mesmo tamanho. O número de bytes impresso é customizável por meio do atributo ```bytes_mostrados```.

In [ ]:
def bits(x):
    addr = id(x)
    n_bytes = sys.getsizeof(x)
    ptr = ctypes.cast(addr, ctypes.POINTER(ctypes.c_uint8))
    bits = ''.join(f'{ptr[i]:08b} ' for i in range(n_bytes))[:-1].split(' ')
    return bits

def comparar(x_bits, y_bits, bytes_mostrados=1):
    n = len(x_bits)
    if bytes_mostrados < n:
        n = bytes_mostrados
    print('   1        2       iguais?')
    for i in range(n):
        print(x_bits[i], y_bits[i], ' ', x_bits[i] == y_bits[i])

Em seguida, geramos alguns objetos e observamos seu conteúdo. No exemplo abaixo, o objeto ```obj2``` é nomeado e salvamos seu conteúdo em bits usando a função acima. Posteriormente, vinculamos o nome ```obj3``` ao mesmo objeto referenciado pelo nome ```obj2```, e de novo salvamos seu conteúdo em bits. Ao final, imprimimos apenas o primeiro byte de cada objeto (contador de referências).

In [ ]:
obj2 = 'b' * 500
antes = bits(obj2)
obj3 = obj2
depois = bits(obj3)
comparar(antes, depois)

   1        2       iguais?
00000010 00000011   False


Observa-se que o contador após o primeiro vínculo conta $2$ (```00000010```) em vez de $1$ (```00000001```)! Por que isso acontece? Porque, como explicado acima, passar um objeto como argumento de uma função também cria um vínculo a ele. Como agora temos nosso código encapsulado na função ```bits()```, o mesmo objeto que havia sido vinculado a ```obj2``` também passa a ser vinculado ao argumento ```x``` e sua contagem de vínculos passa a ser $2$. Em seguida, ao definirmos o nome sinônimo ```obj3```, o contador de vínculos passa a contar $3$ (```00000011```).

Lembremos que $8$ bits permitem contar de $0$ até $256$ em binários. Vamos tentar então dar mais de 256 nomes a um mesmo objeto. Para isso, fazemos abaixo todas as combinações de palavras de duas letras possíveis de se formar com as $17$ primeiras letras do alfabeto (pois $17^2 = 289 > 256$). Vamos dessa vez imprimir os dois primeiros bytes do objeto. Usamos a função ```product``` da biblioteca ```itertools``` para gerar as combinações e entre tais letras, dadas pelo objeto ```string.ascii_uppercase``` (usamos maiúsculas para não correr o risco de sobreescrever comandos ou nomes já definidos).

In [ ]:
from itertools import product
from string import ascii_uppercase

letras = [letra for letra in ascii_uppercase[:17]]
obj4 = 'c' * 500
print('Contador antes:', bits(obj4)[:2])
for permutacao in product(letras, letras):
    nome = permutacao[0] + permutacao[1]
    exec(f'{nome} = obj4')
print('Contador depois:', bits(obj4)[:2])

Contador antes: ['00100001', '00000001']
Contador depois: ['00100011', '00000001']


Vê-se que a primeira contagem foi de $2$, quando tínhamos apenas os vínculos do primeiro nome e do argumento da função ```bits```. Já na segunda contagem, temos o número $35$ no primeiro byte (```00100011```) e o número $2$ no segundo. Ocorre que, na minha arquitetura, o sistema é *little endian*, de modo que o byte mais significativo é o da direita. Por isso, o número todo deve ser lido como $35 + 256 = 291$, que é o que esperaríamos.

O procedimento de lidar com binários é trabalhoso e pode depender da arquitetura do sistema. Por isso, é mais vantajoso usar a função ```sys.getrefcount()```, que nos fornece o número de vínculos de maneira direta. Abaixo damos mais exemplos de contagem, agora testando os mecanismos de desvinculação.

In [ ]:
# contagem de obj4 via método sys.getrefcount()
print('Contagem atual de obj4:', sys.getrefcount(obj4))

# nova contagem após deletar nomes vinculados ao mesmo objeto
del AA
del AB
print('Contagem após duas deleções:', sys.getrefcount(obj4))

# nova contagem após passar o objeto como argumento de uma função
def novo_escopo(obj):
    print('Contagem dentro de um novo escopo:', sys.getrefcount(obj))
novo_escopo(obj4)

# última contagem: após o fechamento do escopo
print('Contagem após fechamento do escopo:', sys.getrefcount(obj4))


Contagem atual de obj4: 291
Contagem após duas deleções: 289
Contagem dentro de um novo escopo: 290
Contagem após fechamento do escopo: 289


### Imortalidade e internamento

Você pode ter notado que os objetos criados na seção anterior parecem desnecessariamente grandes: strings com a mesma letra repetida $500$ vezes. Isso foi proposital, na tentativa de evitar objetos imortais ou internados.

Objetos são **imortais** quando seu contador de vínculos não pode mudar, portanto nunca podem ser desalocados nem coletados pelo coletor de lixo enquanto o interpretador estiver rodando. Esses objetos já nascem criados na memória. Em CPython, alguns singletons ou imutáveis fundamentais são imortais, tais como ```None```, ```True```, ```False```, tipos fundamentais, ```str```s curtas ou comuns, e ```int```s pequenos (geralmente entre $-5$ e $256$).

Um objeto imortal já nasce com um número de vínculos extremamente grande, embora seu valor específico dependa da arquitetura. Para descobrir esse número na arquitetura que você está usando - logo, para saber se um objeto é imortal -, basta chamar a função de contagem de vínculos mencionada acima:

In [ ]:
print('Contagem de vínculos de...')
print('  None:', sys.getrefcount(None))
print('  True:', sys.getrefcount(True))
print('  dict:', sys.getrefcount(dict))
print("  'a':", sys.getrefcount('a'))
print('  1:', sys.getrefcount(1))

Contagem de referências de...
  None: 4294967295
  True: 4294967295
  dict: 4294967295
  'a': 4294967295
  1: 4294967295


Antes, era possível contar o número de vínculos de tais objetos. Porém, desde Python 3.12, isso parou de ser possível devido ao custo de processamento da mudança que era necessário fazer no contador de vínculos do objeto toda vez que uma nova vinculação fosse feita. Se você tiver interesse, a motivação completa para esta mudança e suas consequências podem ser consultadas na página da [PEP 683](https://peps.python.org/pep-0683/). Mas não é preciso ir tão longe para entender um aspecto mais básico: a partir da função ```bits()``` introduzida anteriormente, você pode perceber as repetidas sobrescrições de memória que acontecem a cada nova vinculação. Via de regra, isso ocorre mesmo para objetos imutáveis, o que é uma certa contradição, pois tais objetos efetivamente possuem uma parte de sua memória que é mutável em CPython. Para os imortais, esse já não é mais o caso, permitindo a existência de objetos verdadeiramente imortais em CPython.

Por sua vez, o **internamento** é a criação de um vínculo paralelo ao do nome declarado pelo usuário para objetos não tão grandes. A partir da criação de um vínculo paralelo, o CPython passa a usar preferencialmente este vínculo ao se referir ao objeto. Alguns objetos são automaticamente internados no mesmo momento em que são criados, a exemplo de strings um pouco grandes ou incomuns demais para serem imortais, mas ainda não gigantescas.

In [ ]:
# string enorme: não internada automaticamente
print(sys.getrefcount('0' * 5000))

# string menor: internada automaticamente
print(sys.getrefcount('0' * 3))

# string pequena que parece nome válido: internada automaticamente
print(sys.getrefcount('a1'))


('0' * 5000) is ('0' * 5000)
#


1
2
3


False

6

O motivo pelo qual internamentos e imortais dependem da implementação é que se tratam de mecanismos de otimização. Vamos supor que você usa várias vezes um mesmo objeto imutável e extremamente recorrente. Está claro que é vantajoso guardar apenas uma referência a esse objeto, em vez de usar diversas referências para algo que sequer pode ser alterado. As razões para o 

In [ ]:
a = 'a' * 5000
b = 'a' * 5000
a is b

False

In [ ]:
sys.getrefcount('h' * 5000)

1

### Para além de sinônimos: o operador ```is``` em CPython

Conforme adiantamos acima, o operador ```is``` pode ter ainda mais nuances a depender da implementação.

Quando ambas as expressões retornam o mesmo objeto único (*singletons*) ou imortal, a comparação ```is``` sempre retorna ```True```. Isso pode parecer óbvio, mas às vezes não é fácil de reconhecer para iniciantes na linguagem. Por exemplo, você já se deparou com os casos abaixo?

In [ ]:
# caso 1
x = 5
print('caso 1:', type(x) is int)

# caso 2
u = 1
v = 1
print('caso 2:', u is v)

# caso 3
y = 'a'
z = 'a'
print('caso 3:', y is z)

# caso 4
t500 = 'g' * 500
w500 = 'g' * 500
print('caso 4:', t500 is w500)

# caso 5
t5000 = 'h' * 5000
w5000 = 'h' * 5000
print('caso 5:', t5000 is w5000)

# caso 6
print('caso 6:', 10**9 is 10**9) # numeros grandes nao deveriam ser objetos identicos

caso 1: True
caso 2: True
caso 3: True
caso 4: True
caso 5: False
caso 6: True


<>:26: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
<>:26: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
C:\Users\henri\AppData\Local\Temp\ipykernel_76124\2384360586.py:26: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
  print('caso 6:', 10**9 is 10**9) # numeros grandes nao deveriam ser objetos identicos


O primeiro caso ocorre porque objetos do tipo ```type``` só admitem uma instância, logo o ```int``` retornado por ```type(int)``` é exatamente o mesmo objeto, na memória, do objeto único ```int```. Isso é possível porque, em Python, todo objeto sabe qual é seu tipo. Em CPython, isso é implementado por meio de um ponteiro ao singleton do tipo correspondente (informação que faz parte do conteúdo do objeto).

Já o segundo e o terceiro casos ocorrem porque, em CPython, números inteiros pequenos e expressões reduzidas recebem apenas um endereço de memória, que é apontado por todos os vínculos que usam tal valor (objetos imortais).

Por fim, o quarto caso ocorre porque, de novo na implementação CPython, podem ocorrer otimizações por parte do interpretador ao encontrar expressões que usam sucessivamente valores idênticos, principalmente dentro de uma mesma linha lógica, igualando duas referências. Notar que o Python avisa da inadequação de usar ```is``` para comparar literais.

A partir de Python 3.12, objetos imortais têm uma contagem de referências gigantesca (na implementação do meu computador e da minha versão, $2^{32} - 1$), que não pode ser diminuída, de modo que esses objetos jamais podem ser coletados pelo garbage collector. O mesmo ocorre com alguns singletons essenciais, tais como ```None```, ```True``` e ```False```, ou os tipos fundamentais da linguagem, como os objetos ```int``` e ```str```.

**ESCREVER SOBRE O QUINTO CASO E READEQUAR TEXTO E SUA POSIÇÃO**

In [ ]:
sys.getrefcount([1])



1

## Escopo